In [499]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [500]:
#Tokenization of Question
def tokenization(list_of_strings):
    BOW_Query=list_of_strings.split()
    return BOW_Query

In [501]:
#Removal of Stop words in the Question
def stop_words(strings_list):
    
    stop = set(stopwords.words('english'))
    bow_query_sw=([i for i in strings_list if i not in stop])
    return bow_query_sw

In [502]:
#Stemming of the Question terms
def stemmer(strings_list):
    
    query_words=[]
    ps = PorterStemmer()
    for w in strings_list:
        query_words.append(ps.stem(w))
    return query_words

In [503]:
#Removal of stop words in the answers
def sw_co(comments):    
    stop = set(stopwords.words('english'))
    comments_sw=[]
    for item in comments:
        stop_words=([i for i in item.lower().split() if i not in stop])
        comments_sw.append(stop_words)
    return comments_sw

In [504]:
#Stemming the words in the comments
def stemmer_co(strings):
    comments_st=[]
    ps = PorterStemmer()
    for items in strings:
        comment_st_each=[]
        for w in items:
            comment_st_each.append(ps.stem(w))
        comments_st.append(comment_st_each)
    return comments_st

In [505]:
#Creating the unique words list from the above list.
#Creating a dictionary with word counter
def word_counter(strings):    
    my_counter = {}
    for line in strings:
        for word in line:
            my_counter[word] = my_counter.get(word, 0) + 1
    return my_counter

In [506]:
#Extracting only the words (which corresponds to the dictionary keys)
def unique_words(dictionary):    
    comments_words=[]
    for item in dictionary.keys():
        comments_words.append(item)
    return comments_words

In [507]:
def wordset(querydoc,commentsdoc):
    wordset=set(querydoc).union(set(commentsdoc))
    return wordset

In [508]:
#Initialization with Zeros
def tf_raw_query(query_strings,wordset):    
    worddict_query=dict.fromkeys(wordset,0)    
    for word in query_strings:
        worddict_query[word]+=1
    return worddict_query

In [509]:
#Calculation of the tf-weight terms
def tf_weight(words_raw):    
    dict_tf_wt={}
    import math
    for key,val in words_raw.items():
        if val!=0:
            value=1+(math.log(val))
        else:
            value=0
        dict_tf_wt[key]=value
    return dict_tf_wt

In [510]:
def dfdfdf(ww,dd):
    dict_dfdf={}
    for item in ww:
        #print (item)
        co=0
        for items in Comments_Stemming:
            #print (items)
            if item in items:
                co+=1
            else:
                pass
            dict_dfdf[item]=co
            #print ('First word')
    return dict_dfdf

In [511]:
#Calculating the idf for a query
def idf_query1(dfw):
    idf=[]
    import math
    for item in dfw['df']:
        if item !=0:
            value=math.log(len(Comments_Stemming)/item)
        else:
            value=0
        idf.append(value)
    dfw["idf"]=idf
    return dfw

In [512]:
#Normalizing the weights
def norm_q(df_norm):    
    sum1=0.0
    for item in df_norm['wt']:
        sq=item*item
        sum1+=sq
    import math
    norm_q=[]
    for itemm in df_norm['wt']:
        if itemm==0:
            deno=0
        else:
            deno=itemm/(math.sqrt((sum1)))
        norm_q.append(deno)
    df_norm["nor_q"]=norm_q
    return df_norm

In [513]:
def log_wt(val1):
    import math
    if val1!=0:
        value=1+(math.log(val1))
    else:
        value=0
    return value

#### Importing the Question-Comments XML file,Identifying the Question-Comments within the XML file

In [514]:
#Importing the XML file and required libraries
import xml.etree.ElementTree as ET
import pandas as pd
tree = ET.parse('SemEval2016-Task3-CQA-QL-train-part1-subtaskA.xml')
root = tree.getroot()

In [515]:
scores={}
for thread in root.findall("Thread"):
    dict_score={}
    for question in thread.findall('RelQuestion'):
        Question_ID=question.attrib['RELQ_ID']
        Question_text=question.find('RelQBody').text
        
        print ("Accessing the Question ID ",Question_ID)
        #Tokenization of the Question        
        Tokenized_Question=tokenization(Question_text)
        
        #Removing the stop words from the Question        
        Question_Stop_words=stop_words(Tokenized_Question)        
        
        #Stemming the words in Question        
        Stemmed_question=stemmer(Question_Stop_words)
        
    print ("Accessing the comments for ",Question_ID)  
    
    #Comments
    comments=[]
    
    ids=[]
    for each_answer in thread.findall('RelComment'):
        Answer_ID=each_answer.attrib['RELC_ID']
        Answer_text=each_answer.find('RelCText').text
        comments.append(Answer_text)
        ids.append(Answer_ID)
    
       
    #Removing stop words from the Comments    
    Comments_stop_words=sw_co(comments)
    
    #Stemming the words in the comments
    
    Comments_Stemming=stemmer_co(Comments_stop_words) 
    
    
    #Creating a wordset for the comments
    counter1=word_counter(Comments_Stemming)
    unique_words1=unique_words(counter1)
    wordset1=wordset(Stemmed_question,unique_words1)
    
    
    #Calculating the raw terms in the comments
    tf_raw=tf_raw_query(Stemmed_question,wordset1) 
    tf_wt=tf_weight(tf_raw)
    
    dl=list(wordset1)   
    df_query=dfdfdf(dl,Comments_Stemming)

    #Appending the tf-raw, tf-wt, df into a dataframe
    
    dframe=pd.DataFrame.from_dict([tf_raw,tf_wt,df_query],orient='columns')
    df=dframe.transpose()
    summary_of_results=df.rename(columns={0: 'tf_raw',1: 'tf_weight',2: 'df'})
    
    #Calculating the IDF for the query
    
    idf_query=idf_query1(summary_of_results)    
    idf_query["wt"]=idf_query.idf*idf_query.tf_weight
    #Normalization
    
    norm_query=norm_q(idf_query)
    
    #Calculating the raw-terms for all the comments
    
    result=pd.DataFrame()
    for item in Comments_Stemming:
        worddict_terms=dict.fromkeys(wordset1,0)  
        frames=pd.DataFrame()
        for items in item:
            worddict_terms[items]+=1
            df_com_c1=pd.DataFrame.from_dict([worddict_terms])
        frames=[result,df_com_c1]
        result = pd.concat(frames)
    Comments_raw_terms=result.transpose()
    Comments_weights=Comments_raw_terms.applymap(log_wt)
    
    #Adding the column name to the Dataframe
    Comments_weights.columns=["tf_doc_wt_1","tf_doc_wt_2","tf_doc_wt_3","tf_doc_wt_4","tf_doc_wt_5",
                              "tf_doc_wt_6","tf_doc_wt_7","tf_doc_wt_8","tf_doc_wt_9","tf_doc_wt_10"]

    #Normalization of all the comments weights
    
    counter=1
    for item in Comments_weights:
        sum2=0.0
        for items in Comments_weights[item]:
            sq1=items*items
            sum2+=sq1
            import math
            norm_q_d=[]
            for itemw in Comments_weights[item]:
                if itemw==0:
                    deno=0
                else:
                    deno=itemw/math.sqrt((sum2))
                norm_q_d.append(deno)
        s=str(counter)
        Comments_weights[s]=norm_q_d
        counter+=1
    #Comments_weights
    #Keeping the column weights 
    cols = [0,1,2,3,4,5,6,7,8,9]
    Comments_weights.drop(Comments_weights.columns[cols],axis=1,inplace=True)
    
    #product
    columns = Comments_weights.columns[:]
    for item in columns:
        Comments_weights[item] *= norm_query['nor_q']
        
    #Replacing the NaN values with Zero's
    Comments_weights_corrected=Comments_weights.fillna('0')

    #Query - Documents score
    
    Cosine_Score=Comments_weights_corrected.sum(axis=0)
    scoreee=list(Cosine_Score)
    dictionary = dict(zip(ids, scoreee))
    scores.update(dictionary)
    #print ('printing cosine score')
    #print(dictionary)
    
    for item in dictionary.keys():
        print (Question_ID,item,dictionary[item])
#print (scores)


Accessing the Question ID  Q1_R1
Accessing the comments for  Q1_R1


C:\Users\Jessica\Miniconda3\envs\py35\lib\site-packages\ipykernel\__main__.py:98: RuntimeWarning: divide by zero encountered in double_scalars


Q1_R1 Q1_R1_C5 0.177517449139
Q1_R1 Q1_R1_C4 0.353553390593
Q1_R1 Q1_R1_C2 0.0
Q1_R1 Q1_R1_C8 0.0
Q1_R1 Q1_R1_C7 0.0
Q1_R1 Q1_R1_C6 0.408248290464
Q1_R1 Q1_R1_C9 0.0
Q1_R1 Q1_R1_C1 0.0
Q1_R1 Q1_R1_C3 0.182574185835
Q1_R1 Q1_R1_C10 0.0
Accessing the Question ID  Q1_R6
Accessing the comments for  Q1_R6
Q1_R6 Q1_R6_C1 0.178782666296
Q1_R6 Q1_R6_C8 0.0
Q1_R6 Q1_R6_C7 0.154051929918
Q1_R6 Q1_R6_C2 0.0
Q1_R6 Q1_R6_C9 0.0
Q1_R6 Q1_R6_C10 0.14658464224
Q1_R6 Q1_R6_C5 0.244842060251
Q1_R6 Q1_R6_C4 0.118562670903
Q1_R6 Q1_R6_C6 0.2178623286
Q1_R6 Q1_R6_C3 0.115026140125
Accessing the Question ID  Q1_R8
Accessing the comments for  Q1_R8
Q1_R8 Q1_R8_C5 0.0
Q1_R8 Q1_R8_C8 0.0691945620115
Q1_R8 Q1_R8_C6 0.0963476013073
Q1_R8 Q1_R8_C9 0.0
Q1_R8 Q1_R8_C4 0.263553541628
Q1_R8 Q1_R8_C10 0.0
Q1_R8 Q1_R8_C1 0.0
Q1_R8 Q1_R8_C2 0.109972427526
Q1_R8 Q1_R8_C7 0.0
Q1_R8 Q1_R8_C3 0.0
Accessing the Question ID  Q1_R10
Accessing the comments for  Q1_R10
Q1_R10 Q1_R10_C2 0.0350262872532
Q1_R10 Q1_R10_C5 0.25933278

AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
Cosine_Score

In [ ]:
ids

In [ ]:
scoreee

In [ ]:
print (Question_ID,ids,scoreee)